# CNNでMNIST

認識精度: 98%

モデル: Input => Conv1 => ReLU => Max Pooling 1
                 => Conv2 => ReLU => Max Pooling 2
                 => Full Connected Layer 1 => Dropout => Softmax

In [1]:
# ライブラリ読み込み
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

In [2]:
# データ読み込み
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/mnist', one_hot=True)

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
# 重みwとバイアスbの初期化関数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
# 畳み込み層とプーリング層の関数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                                           strides=[1, 2, 2, 1], padding='SAME')

In [5]:
# モデルの作成
x = tf.placeholder(tf.float32, [None, 784]) # 入力値 Noneはバッチサイズによって実行時に指定
y_ = tf.placeholder(tf.float32, [None, 10]) # 正解ラベル

# 784x1の入力値を、縦28x横28xチャネル1の行列に変換
x_image = tf.reshape(x, [-1,28,28,1]) 

# conv1
W_conv1 = weight_variable([5, 5, 1, 32]) # 縦5x横5xチャネル1のフィルターが32個
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# conv2
W_conv2 = weight_variable([5, 5, 32, 64]) # 縦5x横5xチャネル32のフィルターが64個
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# fc1
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# softmax
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [6]:
# 損失関数
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

In [7]:
# 最適化関数 Adam、Learning Rate 0.0001、損失関数 cross_entropy
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [8]:
# 精度の計算式
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
# トレーニングの実施
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) # 変数の初期化（おまじない）
    
    for step in range(2000): # 2,000回トレーニングを実施
        batch_xs, batch_ys = mnist.train.next_batch(100) # バッチサイズ 100
        _, loss_value, train_acc = sess.run([train_step, cross_entropy, accuracy],
                feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5 }) # placeholderに値セット

        # 進捗を表示
        if step %100 == 0:
            cv_acc = sess.run(accuracy, feed_dict={
                    x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0 })
            print('step %4d, loss = %.2f, train acc = %.3f , cv acc = %.3f' % (
                    step, loss_value, train_acc, cv_acc))
    
    print("test acc %g"%accuracy.eval(feed_dict={
                x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step    0, loss = 14.52, train acc = 0.060 , cv acc = 0.109
step  100, loss = 0.83, train acc = 0.720 , cv acc = 0.873
step  200, loss = 0.53, train acc = 0.810 , cv acc = 0.915
step  300, loss = 0.34, train acc = 0.900 , cv acc = 0.934
step  400, loss = 0.52, train acc = 0.820 , cv acc = 0.944
step  500, loss = 0.16, train acc = 0.940 , cv acc = 0.953
step  600, loss = 0.21, train acc = 0.920 , cv acc = 0.956
step  700, loss = 0.28, train acc = 0.900 , cv acc = 0.960
step  800, loss = 0.30, train acc = 0.900 , cv acc = 0.967
step  900, loss = 0.16, train acc = 0.950 , cv acc = 0.967
step 1000, loss = 0.18, train acc = 0.940 , cv acc = 0.971
step 1100, loss = 0.10, train acc = 0.950 , cv acc = 0.970
step 1200, loss = 0.14, train acc = 0.970 , cv acc = 0.974
step 1300, loss = 0.11, train acc = 0.970 , cv acc = 0.973
step 1400, loss = 0.12, train acc = 0.970 , cv acc = 0.975
step 1500, loss = 0.12, train acc = 0.950 , cv acc = 0.977
step 1600, loss = 0.07, train acc = 0.980 , cv acc = 0.